In [8]:
open Core_kernel

let build_counts () =
  In_channel.fold_lines In_channel.stdin ~init:[] ~f:(fun counts line ->
    let count =
      match List.Assoc.find ~equal:String.equal counts line with
      | None -> 0
      | Some x -> x
    in
    List.Assoc.add ~equal:String.equal counts line (count + 1)
  )

let () =
  build_counts ()
  |> List.sort ~cmp:(fun (_,x) (_,y) -> Int.descending x y)
  |> (fun l -> List.take l 10)
  |> List.iter ~f:(fun (line,count) -> printf "%3d: %s\n" count line)

val build_counts :
  unit -> (Core_kernel.String.t, int) Core_kernel.List.Assoc.t = <fun>


In [5]:
let build_counts () = 
In_channel.fold_lines stdin ~init:[] ~f:(fun counts line -> 
let count = 
match List.Assoc.find counts line with
| None -> 0
| Some x -> x
in
List.Assoc.add counts line (count + 1)
)

File "[5]", line 2, characters 22-27:
Warning 3: deprecated: Core.stdin
[since 2016-04] Use [In_channel.stdin]


error: compile_error

In [6]:
let assoc = [("one", 1); ("two", 2); ("three",3)]

val assoc : (string * int) list = [("one", 1); ("two", 2); ("three", 3)]


In [7]:
List.assoc.find "four" 4

error: compile_error

In [10]:
List.Assoc.add assoc "two"  4 (* overwrite an existing key *) ;;

- : equal:(string -> string -> bool) ->
    (string, int) Core_kernel.List.Assoc.t
= <fun>


In [13]:
let () = print_string "something"

In [14]:
()

- : unit = ()


In [16]:
print_string "few"

- : unit = ()


let is the keyword used to define new variables, like in the following construct:

let pattern = expr

assigns the value 2 to the name a. (Note this is not a way to assign a value to an already existing variable, but this is another topic).

But the pattern to the left of the = sign can be more than just a name. For instance

In [17]:
let (a,b) = (42,"foo")

val a : int = 42
val b : string = "foo"


defines both a and b, to be respectively 42 and "foo". Of course, the types on both sides must match. Which is the case here: both sides are of type int * string.

The expressions to the right of the = sign can also be elaborated, for instance

defines both a and b, to be respectively 42 and "foo". Of course, the types on both sides must match. Which is the case here: both sides are of type int * string.

The expressions to the right of the = sign can also be elaborated, for instance

In [18]:
let foo =
  let temp = String.make 10 'a' in
  temp.[2] <- 'b';
  temp

File "[18]", line 3, characters 2-17:
Warning 3: deprecated: Core_kernel.String.set
[since 2017-10] Use [Bytes.set] instead


val foo : Core_kernel.String.t = "aabaaaaaaa"


defines foo as the string "aabaaaaaaa". (As a side note, it also ensures that temp is local to this code snippet).

Now, let's use both: on the left, a pattern matching values of type unit, and on the right, an expression of type unit:

In [19]:
let () = Printf.printf "Hello world!\n"

Which explains the let () = construct. Now, about the let _, one simply needs to know that _ can be used in a pattern as a wildcard: it matches values of any type and does not bind any name. For instance

In [20]:
let (a,_) = (42,"foo")

val a : int = 42


defines a as 42, and discards the value "foo". _ means "I know there is something here and I explicitly say I will not use it, so I don't name it". Here _ was used to match values of type string, but it can match value of any type, like int * string:

which does not define any variable and is not very useful. Such constructs are useful when the right hand side has side effects, like this:

In [21]:
let _ = Printf.printf "Hello world!\n"

- : unit = ()


which explains the second part of the question.

Both are used and it's rather a matter of taste whether to use one or the other.

let () = is slightly safer as it has the compiler check that the right hand side is of type unit. A value of any other type than unit is often a bug.

let _ = is slightly shorter (I've seen this argument). (Note that with an editor that automatically closes parenthesizes, the number of keystrokes is the same ;-)

与`C`不同，`Ocaml`中程序并没有一个唯一的`main`函数。执行`Ocaml`程序的时候，实现文件中的所有语句会暗器链接的顺序进行计算。这些实现文件可能包含任意的表达式，而不只是函数定义。在这个例子。在这个例子中，以`let()=`开头的声明就扮演着`main`函数的角色，启动了处理过程。不过，整个文件都会在启动时计算，所以在某种程度来看，整个代码基都是一个庞大的`main`函数。

写`let ()=`是一个惯用法，看上去有点奇怪，不过这样有一个目的。这里的`let`绑定是对一个`unit`类型值的模式匹配，它是为了确保右边的表达式返回`unit`，对于主要为得到副作用的函数来说，这个用法很常见。

`Ocaml`附带了两个编译器：`ocamlc`字节码编译器和`ocamlopt`原生代码编译器。用`ocamlc`编译的程序将由一个虚拟机解释，而用`ocamlopt`编译的程序会编译为在一个特定的操作系统和处理器体系结构上运行的原生机器代码。使用`ocamlbuild`便宜时，以`byte`结尾的目标会构建为字节码可执行程序，以`.native`结尾的目标将构建为原生代码。

除了性能以外，这两个编译器生成的可执行程序的行为基本上完全相同。有几点需要注意：首先，字节码编译器可以在更多体系上使用，而且字节码编译器的一些工具并不适用原生代码。例如，Ocaml调试工具只能处理字节码[不过，(gdb)确实可以处理Ocaml原生代码应用]。字节码编译器比原生代码编译器速度更快。另外，要运行一个字节码可执行程序，通过需要在当前系统上安装Ocaml。不过